In [2]:
import glob
import paddle
from paddle import nn
from paddle.static import InputSpec
from paddle.io import Dataset, DataLoader, random_split
import paddle.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [3]:
from def_for_cls import sreen_c_data,Class_bert_NN

In [4]:
tg = 'CHEMBL4896'

In [5]:
# 获取筛选后的预处理地址
screening_pth = f'./fingered_s_data/{tg}_s_c_fg.csv'
df = pd.read_csv(screening_pth)
df_index = df.iloc[:, 0]

In [6]:
# 数据集加载
data_of_desc = sreen_c_data(screening_pth) #tg为选择靶点
descriptor_size = data_of_desc.feature_size

In [7]:
#划分筛选集
screen_loader = DataLoader(data_of_desc, batch_size=64, shuffle=False)

In [8]:
# 实例化模型
num_classes = 2  # 二分类模型[0,1]即无活性和有活性
model = Class_bert_NN(tg,descriptor_size , num_classes=num_classes).to('gpu:0')
#加载
model.set_state_dict(paddle.load(f'./model/{tg}_classify_best_model.pdparams'))

[2024-10-06 20:02:13,589] [ WARNING] module_wrapper.py:149 - From C:\ProgramData\anaconda3\envs\C4-AI\lib\site-packages\keras\src\backend\common\global_state.py:82: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 157ms/step


([], [])

In [9]:
put_out0=[]
#进行筛选
for batch_id, descriptors in enumerate(screen_loader):
    # 前向传播
    predictions = model(descriptors)
    put_out0.append(predictions)
put_out = paddle.concat(put_out0, axis=0)
#处理成概率
put_out = F.softmax(put_out, axis=1)
put_out=np.asarray(put_out.numpy())#转化为np数组
put_out = put_out[:, 1]

In [10]:
# 使用 concat 拼接
result = pd.concat([df_index, pd.Series(put_out)], axis=1)

In [11]:
# 保存到CSV文件，不包含索引
result.to_csv('./screened/{tg}_s_c.csv', index=False)

In [12]:
return result

SyntaxError: 'return' outside function (966808787.py, line 1)